<a href="https://colab.research.google.com/github/MaggieWelch/labs/blob/main/MW_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Pick something else on Craigslist: Musical instruments, roommates, antiques, etc. Look at the search page and its source code. Record which fields/data you would like to gather, and what kinds of EDA you'd do with it.

I used furniture


In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import requests

header = {"User Agent":'MacBook-Air-3:~ maggiewelch$'}
url = 'https://charlottesville.craigslist.org/search/fua#search=1~gallery~0~0'
raw = requests.get(url,headers=header)

2. Get your search results of interest and extract data from them, using code similar to what's below.


In [12]:
from bs4 import BeautifulSoup as soup
bsObj = soup(raw.content,'html.parser')
listings = bsObj.find_all(class_="cl-static-search-result")

In [18]:
import re

furniture_type = ['chair','bench','desk','cabinet','table','stool','shelf','dresser','frame','lamp','stand' ]

data = [] # We'll save our listings in this object
for k in range( len(listings) ):
    title = listings[k].find('div',class_='title').get_text().lower()
    price = listings[k].find('div',class_='price').get_text()
    link = listings[k].find(href=True)['href']
    # Get furniture type from the title string:
    words = title.split()
    hits = [word for word in words if word in furniture_type] # Finds furniture types in the title
    if len(hits) == 0:
        furniture_type = 'missing'
    else:
        furniture_type = hits[0]
    # Get years from title string:
    regex_search = re.search(r'20[0-9][0-9]|19[0-9][0-9]', title ) # Find year references
    if regex_search is None: # If no hits, record year as missing value
        year = np.nan
    else: # If hits, record year as first match
        year = regex_search.group(0)
    #
    data.append({'title':title,'price':price,'link':link,'type':furniture_type})

3. Wrangle your data, do some EDA, and make some plots. Try to find some interesting relationships or stories to tell about your data.


In [21]:
## Wrangle the data
df = pd.DataFrame.from_dict(data)
df['price'] = df['price'].str.replace('$','')
df['price'] = df['price'].str.replace(',','')
df['price'] = pd.to_numeric(df['price'],errors='coerce')
print(df.shape)
df.to_csv('craigslist_cville_furniture.csv') # Save data in case of a disaster
df.head()

(360, 4)


,title,price,link,type
0,red oak live edge bench,160,https://charlottesville.craigslist.org/fuo/d/c...,bench
1,great condition kitchen cabinet for sale $200 ...,200,https://charlottesville.craigslist.org/fuo/d/c...,bench
2,great condition chair $40 or best offer,40,https://charlottesville.craigslist.org/fuo/d/c...,bench
3,2pc desk,59,https://charlottesville.craigslist.org/fuo/d/b...,bench
4,ladies' round back chair,35,https://charlottesville.craigslist.org/fuo/d/c...,bench
